# Viabilidade técnica de extração de informação dos Comprovantes PDF 

## Hipótese

A principal hipose é que *Podemos Extrair* o custo e o CFP/CNPJ a partir dos comprovantes submetidos pelos proponentes.

O *Resultado Esperado* é que, para cada comprovante submetido, conseguimos extrair o CNPJ/CPF do fornecedor, além do custo empenhado pela nota fiscal

*Teremos sucesso quando* conseguirmos extrair tais informações de pelo menos 50% dos comprovantes. 

## Solução Proposta

A solução proposta usa a OCR Tesseract (https://github.com/tesseract-ocr) para a extração de texto a partir de imagens. Para isso a seguinte sequencia de processamento foi necessário:
* [ x ] Querbrar os pdf para ter um comprovante por pdf
* [x] Converter os arquivos pdf em arquivos de imagem
* [x] Fazer o mínimo de processamento de imagem para melhorar a qualidade dos comprovantes
* [x] Extrair o texto a partir da imagem, utilizando o Tesseract
* [x] Extrair por meio de regex o CNPJ/CPF presente no arquivo texto extraído
* [ ] Extrair por meio de regex o valor empenhado na nota fiscal

### 1. Converter pdf em arquivo texto

In [48]:
import argparse
import cv2
import pytesseract
import os
import glob
from PIL import Image, ImageEnhance, ImageFilter

foldernames = sorted(glob.glob('./*/'))
len(foldernames)
for f in foldernames:
    foldersearch= f + "*.jpg"
    filenames = sorted(glob.glob(foldersearch)) 
    i = 0
    for file in filenames:
        print(file)
        im = Image.open(file)
        im = im.filter(ImageFilter.MedianFilter())
        enhancer = ImageEnhance.Contrast(im)
        im = enhancer.enhance(2)
        im = im.convert('1')
        im.save(file)
        text = pytesseract.image_to_string(Image.open(file), lang='por')
        
        os.remove(file)
        print(text)
        text_file = open( f+"%s.txt" % i , "w")
        i += 1
        text_file.write(text)
        text_file.close()

./1613119/document-page0.jpg
LATAM

AIRLINES

TAM Linhas Aéreas S/A

"%

Rua Verbo Divino Nº 2001, andares: 3º conj. 31 e 32,4º conj. 41 e 42, 5"cong. 51 e 52, 6º conj. 61 e 62, 8º conj. B1 e £2,
10º conj. 101 e 102, 12º conj. 121 e 122,13º conj. 131 e 132,15º conj. 151,16º conj. 161, 17*º cong. 171 e 172,

Edifício Condomínio Espaço Empresarial Nações Unidas. Chácara Santo Antonio, CEP 047 19-002, São Paulo / SP

CNPJ (M.F.) Nº D2.012.662/0D001-60 - INSCR. Est. Nº 112.347.127.116

Informação sobre sua passagem

Este é o seu comprovante de compra e contém os detalhes do serviço adquirido.

Nodtadasuamgem não é necessario leva-lo com você.
Tenha em mãos apenas o código de reserva e a documentação de embarque.

ILANA BAR NISSIM WIRGUES

KCUGKO
Adulto

  

Descrição do pagamento

 

Tarlfa equivalente em moeda de BRO ds I0000DCCCOOLIN2O 601667
W '
Taxas etou impoetos (2) BRL S5,84 () BRL: Reais

a Detalhe ttas taus etou tmpostos, Passagent
Totai passagem 3S71-2T170ASSASA BRL 6S3,64 BRL BR

R

Al

 

e 82,

. 81
. 71 e172,

conj,

81 e 62, &º
161, 17º

conj.

. 51 e 62, &*

FT 19-002, São Paulo / SP

01-80 - INSCR. Est. Nº 112347 127 116

conj

- 151, 165º

 

conj

 

 

mg (

M

l
l

U

en
l

U

Y

U

Â

1

ê
É

W
m

| e

o

ú

(AA

J
1
j
n
o
|

l
S
o

tempo

ranta

 

Qquando Tor fazes a SuUa Meserve em até 48

operar no aeroporto de Guaruíhos (SRU), Congonhas

     

FUSAS

  

 

         

                         

b
é
Ã

  

 

    

d
J

   

               

   

 

ú

 

 

13

ertre 05 al

rido.

o

U

Pauto, pod
ente eles.
18

1

conmn.

31 e 4
121 e 122, 13º

conj

3
n
S
=
Ê

Rua Vert

41 e 42, 5º
. 131 e 132, 15º

ial Nações Unidas. Chacara Santo Antonio, CEP |

conj

conj.
o Espaço E

i=

&
—
M,
EJ
e
&

1

cio

  

10º

m

 

| e contém os detalhes do serviço

. não é necessario leva-lo com você.

te de

CNPJ (M.F.) Nº 02012 862
ado

: Em

s
É&
Ê
&
&
&
S

"-
m

Z

 

e a documentação

| OTAVIO |

o código

 

ú
a

|

1

 

e

 

 

 

 

M
b

        

 



TesseractError: (-6, 'Tesseract Open Source OCR Engine v4.0.0-beta.2-313-g29f28 with Leptonica Warning. Invalid resolution 0 dpi. Using 70 instead. Estimating resolution as 532 contains_unichar_id(unichar_id):Error:Assert failed:in file ../../src/ccutil/unicharset.h, line 511')

### 2. Extrair CNPJ a partir do texto

In [46]:
import re
import argparse
import cv2
import pytesseract
import os
import glob
from PIL import Image, ImageEnhance, ImageFilter

p = re.compile('\d{2}\.\d{3}\.\d{3}\/\d{4}-\d{2}')
foldernames = sorted(glob.glob('./*/'))
print(len(foldernames))
i = 0
for f in foldernames:
    foldersearch= f + "*.txt"
    filenames = sorted(glob.glob(foldersearch)) 
    for file in filenames:
        text_file = open( file , "r")
        for line in text_file:
            s = p.match(line)
            if s:
                print(s.group(), i)
                i += 1

100
13.631.755/0001-36 0
13.631.755/0001-36 1
13.631.755/0001-36 2
09.272.200/0001-78 3
18.659.166/0001-80 4
09.272.200/0001-78 5
18.659.166/0001-80 6
09.272.200/0001-78 7
18.659.166/0001-80 8
09.272.200/0001-78 9
18.659.166/0001-80 10
18.659.166/0001-80 11
07.850.254/0001-48 12
09.272.200/0001-78 13
18.659.166/0001-80 14
18.659.166/0001-80 15
07.850.254/0001-48 16
18.659.166/0001-80 17
07.850.254/0001-48 18
07.850.254/0001-48 19
18.659.166/0001-80 20
09.272.200/0001-78 21
18.659.166/0001-80 22
28.398.653/0001-90 23
28.398.653/0001-90 24
22.762.192/0001-06 25
00.703.068/0001-37 26
00.000.644/4358-28 27


### 3. Extrair Valor empenhado a partir do texto

## Resultados e Analise

Para um primeiro experimento, conseguimos extrair de forma automatizada somente 30% dos CNPJs dos comprovantes, o que faz com que nao tenhamos atingido nossa meta de sucesso. Logo, há necessidade de evoluir principalmente o algoritmo de processamento de imagem para melhorar a qualidade dos comprovantes

## Próximos passos

* [] Validar padrões regex nos arquivos pdf
* [] extrair preços, a partir de regex
* [] melhorar o processamento de imagem (filtros, contraste, orientação do scanner)
* [] testar algoritmos de extraçao de texto a partir de arquivos pdf nativos (&& apt-get -y install poppler-utils )
